# Room Noise Characterisation

In [18]:
import numpy as np
import matplotlib.pyplot as plt
import math
import pyroomacoustics as pra
from pyroomacoustics.directivities import (
    DirectivityPattern,
    DirectionVector,
    CardioidFamily,
)
from scipy.io import wavfile
from scipy import signal
from playsound import playsound

In [24]:
# Load recorded array signals
n_mics = 8
N = 1024
fs = 48000

mic_signals_filepath = '../array_samples/twosource_room/0'
signals_suffix = '-241005_1739.wav'

for i in range(n_mics):
    file_str = mic_signals_filepath + str(i+1) + signals_suffix
    f_sample, audio = wavfile.read(file_str)

    if i == 0:
        mic_signals = np.zeros((n_mics, len(audio)))

    mic_signals[n_mics-1-i, :] = audio
print(mic_signals.shape)

(8, 872224)


/var/folders/_f/h8xwd2k15l36zgym21rpns740000gn/T/ipykernel_48377/2127467484.py:11: WavFileWarning: Chunk (non-data) not understood, skipping it.
  f_sample, audio = wavfile.read(file_str)


In [25]:
# noise covariance computation

for i in range(n_mics):
    # STFT of signal in non-overlapping segments
    stft = pra.transform.stft.STFT(N, hop=N) # window of 1024 samples, moving in 1024 increments - roughly 20ms long
    X = stft.analysis(mic_signals[1,:]).T
    X_mag = abs(X)

    # alternative STFT with scipy
    # window = signal.windows.hann(N//2, False)
    # SFT = signal.ShortTimeFFT(window, hop=N, fs=fs, mfft=N, scale_to='magnitude')
    # X_check = SFT.stft(mic_signals[1,:])
    # print(abs(X_check))

    # average over different samples
    X_mag_avg = np.mean(X_mag, axis=1)
    print(X_mag_avg.shape)

(513,)
(513,)
(513,)
(513,)
(513,)
(513,)
(513,)
(513,)


In [26]:
# time domain covariance
snapshot = mic_signals[:, :]

R_n = (1.0/snapshot.shape[1]) * np.cov(snapshot)
print(R_n)

[[20654616.67361348 20853777.35752959 -1806615.68615707  4434510.54805085
  17098811.67104688 17176958.74636023  7569512.09324695 11088844.48604913]
 [20853777.35752959 24254335.6018059  -3170560.9428739   4274536.40960728
  19087873.08377382 19476884.58633859  7633524.78225087 13069792.12077625]
 [-1806615.68615707 -3170560.9428739   1932154.73639526    76418.01201197
  -2172365.98675381 -2299102.04576704  1209572.20645598  -747224.8825734 ]
 [ 4434510.54805085  4274536.40960728    76418.01201197  1147460.76092441
   3644878.83984727  3639996.18505778  2326827.66914054  2627741.64885794]
 [17098811.67104688 19087873.08377382 -2172365.98675381  3644878.83984727
  15379137.22296358 15600003.06103821  6727424.5103848  10471425.01964006]
 [17176958.74636023 19476884.58633859 -2299102.04576704  3639996.18505778
  15600003.06103821 15942467.3266899   6927256.38703477 10792236.44595378]
 [ 7569512.09324695  7633524.78225087  1209572.20645598  2326827.66914054
   6727424.5103848   6927256.387